<a href="https://colab.research.google.com/github/DARWICHAli/notebooks/blob/main/comfort_zone.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

using pytorch, will train a model with any architecture , the model deduct from the data when reaching conversion, in order to apply extra attention to the neglected classes, and accelerate computation time

* [ ] train loop
* [ ] scheduler
* [ ] conversion detectin function
* [ ] detection on train loop




In [ ]:
!pip install torchmetrics


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 805.2/805.2 kB 10.0 MB/s eta 0:00:00


In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
from torchvision import datasets, models, transforms
import numpy as np
import matplotlib.pyplot as plt
import cv2
import time
import os
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import confusion_matrix
from torchmetrics.classification import ROC , AUROC
import seaborn as sns
from torch.utils.tensorboard import SummaryWriter
from torchvision.models import ResNet18_Weights
import argparse
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch.utils.data import DataLoader, SubsetRandomSampler
import copy
import random
import json
import time
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu") # device object


In [ ]:
transforms_train = transforms.Compose([
    transforms.RandomResizedCrop(224),
    transforms.RandomHorizontalFlip(),
    transforms.RandomVerticalFlip(),
    transforms.ColorJitter(),
    transforms.RandomRotation(degrees=15),
    transforms.RandomAffine(degrees=15),
    transforms.RandomPerspective(),
    #transforms.RandomErasing(p=1, scale=(0.02, 0.33), ratio=(0.3, 3.3), value=0, inplace=False),
    transforms.GaussianBlur(kernel_size=3),
    #transforms.Cutout(num_holes=1, max_h_size=32, max_w_size=32),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])

transforms_test = transforms.Compose([
                                    transforms.Resize((224, 224)),   #must same as here
                                    transforms.ToTensor(),
                                    transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
])


In [ ]:
def conf_matrix(test_loader,model,mode,arch,class_names,epochs=0):
    # Evaluate the model on test data
    predictions = []
    targets = []
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            #pred = output.argmax(dim=1, keepdim=True)
            pred = torch.argmax(torch.softmax(output,dim=1),1)
            predictions.extend(pred.cpu().numpy())
            targets.extend(target.cpu().numpy())

    # Create the confusion matrix
    cm = confusion_matrix(targets, predictions)

    # Plot the confusion matrix using Seaborn
    sns.set()
    plt.figure(figsize=(12,12))
    sns.heatmap(cm, annot=False, cmap="Blues", xticklabels=class_names, yticklabels=class_names)
    #sns.heatmap(cm, annot=False, cmap="Blues")
    plt.xlabel("Predicted Labels")
    plt.ylabel("True Labels")
    #plt.show()
    # Save the plot as a PNG file
    os.makedirs("results",exist_ok=True)
    plt.savefig(os.path.join( "results",f'confusion_matrix_{arch}_{mode}_in_{epochs}.png'), dpi=300, bbox_inches='tight')


In [ ]:
def roc_plot(test_loader,model,epochs,num_class,mode,arch):
    roc = ROC(task="multiclass",num_classes=num_class)
    predictions = []
    targets = []
    model.eval()
    with torch.no_grad():
        for data, target in test_loader:
            data = data.to(device)
            target = target.to(device)
            output = model(data)
            pred = torch.softmax(output,dim=1)
            predictions.extend(pred.cpu().numpy())
            targets.extend(target.cpu().numpy())

    pred = torch.FloatTensor(predictions)
    targ = torch.IntTensor(targets)
    auroc = AUROC(task="multiclass",num_classes=num_class,average='macro')

    fpr, tpr, _ = roc(pred, targ)

    # plot ROC curves
    plt.figure(figsize=(15, 15))
    plt.plot([0, 1], [0, 1], 'k--')
    for i in range(num_class):
        plt.plot(fpr[i], tpr[i], label='Class {}'.format(i))
    plt.xlabel('False positive rate')
    plt.ylabel('True positive rate')
    plt.title('ROC curve for multi-class classification with AUC= {:.4f}'.format(auroc(pred,targ)))
    plt.legend()
    #plt.show()
    os.makedirs("results",exist_ok=True)
    plt.savefig(os.path.join( "results",f'ROC_in_{arch}_{mode}_{epochs}.png'), dpi=300, bbox_inches='tight')


In [ ]:
def load_dataset(train_dir):
    #################################   dataset   ############################

    train_path = os.path.join(train_dir,'train')
    val_path = os.path.join(train_dir,'val')


    train_dataset = datasets.ImageFolder(train_path, transforms_train)
    val_dataset = datasets.ImageFolder(val_path, transforms_test)
    # Define the split ratio
    #split_ratio = 0.8  # 80% for training, 20% for validation

    # Calculate the sizes of the training and validation sets based on the split ratio
    #train_size = int(len(train_dataset) * split_ratio)
    #val_size = len(train_dataset) - train_size
    #train_set, test_set = torch.utils.data.random_split(train_dataset, [train_size, val_size])
    train_dataloader = torch.utils.data.DataLoader(train_dataset, batch_size=20, shuffle=True, num_workers=4, pin_memory=True)
    test_dataloader = torch.utils.data.DataLoader(val_dataset, batch_size=20, shuffle=False, num_workers=4)

    class_names = train_dataset.classes
    num_classes = len(class_names)
    class_weights = compute_class_weight(class_weight='balanced', classes=list(set(train_dataset.targets)), y=train_dataset.targets)
    tensor_weights = torch.FloatTensor(class_weights)

    return train_dataloader,train_dataset,test_dataloader,train_dataset,num_classes,tensor_weights,class_names

In [ ]:

def train(model,best,mode,train_dataloader,optimizer,criterion,train_dataset,test_dataloader,path,writer,epochs):

    class_num = len(train_dataset.classes)
    num_epochs = epochs   #(set no of epochs)
    start_time = time.time() #(for showing time)
    scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=5,verbose=True)

    cur_epoch = best #to save final epoch if it doesn't enter to avoid crash(outside the loop)
    for epoch in range(best,num_epochs+1): #(loop for every epoch)
        cur_epoch = epoch
        print("Epoch {} running".format(epoch)) #(printing message)
        """ Training Phase """
        model.train()    #(training model)
        running_loss = 0.   #(set loss 0)
        running_corrects = 0
        running_corrects_5 = 0

        # load a batch data of images
        for i, (inputs, labels) in enumerate(train_dataloader):
            inputs = inputs.to(device)
            labels = labels.to(device)
            # forward inputs and get output
            optimizer.zero_grad()
            outputs = model(inputs)
            #_, preds = torch.max(outputs, 1)
            preds = torch.argmax(torch.softmax(outputs, dim=1), 1)
            _, top5_indices = torch.topk(outputs.softmax(dim=-1), k=5)

            loss = criterion(outputs, labels)
            # get loss value and update the network weights
            loss.backward()
            optimizer.step()
            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels.data)
            running_corrects_5 += torch.sum(torch.any(top5_indices == labels.data.view(-1, 1), dim=1))

        epoch_loss = running_loss / len(train_dataloader.dataset)
        epoch_acc = running_corrects / len(train_dataloader.dataset) * 100.
        epoch_acc_5 = (running_corrects_5 / len(train_dataloader.dataset)) * 100.
        print('[Train #{}] Loss: {:.4f} TOP1ACC: {:.4f}% TOP5ACC: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc,epoch_acc_5, time.time() - start_time))
        writer.add_scalar('Training Loss', epoch_loss, epoch)
        writer.add_scalar('Training Accuracy TOP1', epoch_acc, epoch)
        writer.add_scalar('Training Accuracy TOP5', epoch_acc_5, epoch)


        """ Testing Phase """
        with torch.inference_mode():
            model.eval()
            running_loss = 0.
            running_corrects = 0
            running_corrects_5 = 0.

            for inputs, labels in test_dataloader:
                inputs = inputs.to(device)
                labels = labels.to(device)
                outputs = model(inputs)
                preds = torch.argmax(torch.softmax(outputs, dim=1), 1)
                _, top5_indices = torch.topk(outputs.softmax(dim=-1), k=5)
                loss = criterion(outputs, labels)
                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds == labels.data)
                running_corrects_5 += torch.sum(torch.any(top5_indices == labels.data.view(-1, 1), dim=1))

            epoch_loss = running_loss / len(test_dataloader.dataset)
            epoch_acc = (running_corrects / len(test_dataloader.dataset)) * 100.
            epoch_acc_5 = (running_corrects_5 / len(test_dataloader.dataset)) * 100.
            print('[Test #{}] Loss: {:.4f} TOP1ACC: {:.4f}% TOP5ACC: {:.4f}% Time: {:.4f}s'.format(epoch, epoch_loss, epoch_acc,epoch_acc_5, time.time() - start_time))
            writer.add_scalar('Testing Loss', epoch_loss, epoch)
            writer.add_scalar('Testing Accuracy TOP1', epoch_acc, epoch)
            writer.add_scalar('Testing Accuracy TOP5', epoch_acc_5, epoch)
            scheduler.step(epoch_loss)
        if epoch_loss < min_loss :
            min_loss = epoch_loss
            conf_matrix(test_dataloader,model,mode,arch,train_dataset.classes,epoch)
            roc_plot(test_dataloader,model,epoch,class_num,mode,arch)




    torch.save({
    'epoch': cur_epoch,
    'model_state_dict': model.state_dict(),
    'optimizer_state_dict': optimizer.state_dict(),
    'loss': epoch_loss
    }, "model.pth")
    conf_matrix(test_dataloader,model,mode,arch,train_dataset.classes,cur_epoch)
    roc_plot(test_dataloader,model,cur_epoch,class_num,mode,arch)


    return model

In [ ]:
#def train_cz(model,best,dataset,dataloader,optimizer,criterion,path,writer,total_epochs):

def train_cz(model,dataset,dataloader,optimizer,criterion,writer,total_epochs):
    # Convergence parameters
    min_loss_change = 1e-3  # Minimum change in loss to consider convergence
    patience = 5  # Number of epochs with no improvement after which to consider convergence
    last_10_values = []
    best_model = copy.deepcopy(model)
    best_loss = float('inf')
    no_improvement_count = 0
    scheduler = ReduceLROnPlateau(optimizer, factor=0.1, patience=10,verbose=True)
    start_time = time.time()
    for epoch in range(total_epochs):

        total_loss = 0.0
        running_corrects = 0
        running_corrects_5 = 0
        for batch_idx, (inputs, targets) in enumerate(dataloader):
            optimizer.zero_grad()
            inputs = inputs.to(device)
            targets = targets.to(device)
            outputs = model(inputs)
            loss = criterion(outputs, targets)
            loss.backward()
            optimizer.step()
            preds = torch.argmax(torch.softmax(outputs, dim=1), 1)
            _, top5_indices = torch.topk(outputs.softmax(dim=-1), k=5)

            running_corrects += torch.sum(preds == targets.data)
            running_corrects_5 += torch.sum(torch.any(top5_indices == targets.data.view(-1, 1), dim=1))
            epoch_acc = (running_corrects / len(dataloader.dataset)) * 100.
            epoch_acc_5 = (running_corrects_5 / len(dataloader.dataset)) * 100.



            # Calculate total loss for the epoch
            total_loss += loss.item()

        # Calculate average loss for the epoch
        avg_loss = total_loss / len(dataloader)
        scheduler.step(avg_loss)



        # Detect convergence based on change in loss
        if epoch > 0:
            if len(last_10_values) > 10:
                    last_10_values.pop(0)

            loss_change = avg_loss - prev_loss
            if loss_change < min_loss_change:
                #no_improvement_count += 1
                last_10_values.append(-1)
                print(f"no improvement, no_improvement_count = {last_10_values.count(-1)}")
            else:
                #no_improvement_count = 0
                last_10_values.append(0)

            if last_10_values.count(-1) >= patience:
                last_10_values = []
                print(f'Converged after {epoch + 1} epochs.')
                # Modify batch composition to prioritize best-performing class
                best_class_indices = (targets == torch.argmax(outputs, dim=1)).nonzero().squeeze()
                random.shuffle(best_class_indices)
                half_size = len(best_class_indices) // 2
                subset_indices = best_class_indices[:half_size]
                subset_sampler = SubsetRandomSampler(subset_indices)
                #dataloader = DataLoader(dataset, batch_size=32, sampler=subset_sampler)
                dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, sampler=subset_sampler, num_workers=2, pin_memory=True)
                #break

        # Update best model if the current model has a lower loss
        if avg_loss < best_loss:
            best_loss = avg_loss
            best_model = copy.deepcopy(model)

        prev_loss = avg_loss
        # Print training statistics

        print('[Train #{}] Loss: {:.4f} TOP1ACC: {:.4f}% TOP5ACC: {:.4f}% Time: {:.4f}s'.format(epoch, avg_loss, epoch_acc,epoch_acc_5, time.time() - start_time))
        #print(f'Epoch [{epoch + 1}/{total_epochs}], Average Loss: {avg_loss:.4f}')

    # Training completed
    print('Training finished.')
    return best_model


In [ ]:
# CIFAR-10 classes
class_names = ('plane', 'car', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

epochs = 500

# Load the CIFAR-10 training and test datasets
dataset = datasets.CIFAR10(root='./data', train=False, download=True, transform=transforms_train)

# Create data loaders for training and test datasets
dataloader = torch.utils.data.DataLoader(dataset, batch_size=64, shuffle=True, num_workers=2, pin_memory=True)

class_names = dataset.classes
num_classes = len(class_names)
class_weights = compute_class_weight(class_weight='balanced', classes=list(set(dataset.targets)), y=dataset.targets)
tensor_weights = torch.FloatTensor(class_weights)



model = models.resnet18(ResNet18_Weights.DEFAULT)   #load resnet18 model

for name ,param in model.named_parameters():
    if 'layer4' in name :
        param.requires_grad = True # False
    else :
        param.requires_grad = False # False
num_features = model.fc.in_features     #extract fc layers features
model.fc = nn.Sequential(nn.Linear(num_features, len(class_names)))
model = model.to(device)

tensor_weights = tensor_weights.to(device)
criterion = nn.CrossEntropyLoss(weight = tensor_weights)

optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9,weight_decay=0.0001)
#optimizer = optim.Adam(model.parameters(), lr=0.05)

log_dir = os.path.join("logs",f"resnet18")

#tmp = optimizer.param_groups[0]['lr']
#optimizer.param_groups[0]['lr'] = 0.0001
tmp = optimizer.param_groups[0]['lr']
print(f'training resnet18 for {epochs}. Current lr is {tmp}.')
writer = SummaryWriter(log_dir)
model = train_cz(model,dataset,dataloader,optimizer,criterion,writer,epochs)
writer.close()


Files already downloaded and verified


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:135: UserWarning: Using 'weights' as positional parameter(s) is deprecated since 0.13 and may be removed in the future. Please use keyword parameter(s) instead.
  warnings.warn(


training resnet18 for 500. Current lr is 0.01.
[Test #0] Loss: 1.6034 TOP1ACC: 42.4600% TOP5ACC: 88.3700% Time: 66.4556s
[Test #1] Loss: 1.3746 TOP1ACC: 51.7900% TOP5ACC: 92.8800% Time: 136.2132s
[Test #2] Loss: 1.2964 TOP1ACC: 54.7300% TOP5ACC: 93.5100% Time: 203.2596s
[Test #3] Loss: 1.2352 TOP1ACC: 57.1600% TOP5ACC: 94.2000% Time: 268.3040s
[Test #4] Loss: 1.1775 TOP1ACC: 58.4700% TOP5ACC: 94.8800% Time: 334.0584s
[Test #5] Loss: 1.1642 TOP1ACC: 58.9100% TOP5ACC: 95.1700% Time: 401.0149s
[Test #6] Loss: 1.1333 TOP1ACC: 60.4200% TOP5ACC: 95.3000% Time: 468.6760s
[Test #7] Loss: 1.0971 TOP1ACC: 60.8000% TOP5ACC: 95.5000% Time: 535.9012s
[Test #8] Loss: 1.0740 TOP1ACC: 62.1600% TOP5ACC: 95.7700% Time: 602.9630s
[Test #9] Loss: 1.0600 TOP1ACC: 62.9300% TOP5ACC: 95.7000% Time: 671.7839s
[Test #10] Loss: 1.0439 TOP1ACC: 63.7800% TOP5ACC: 95.9500% Time: 739.8687s
[Test #11] Loss: 1.0379 TOP1ACC: 62.7800% TOP5ACC: 95.9000% Time: 806.4990s
[Test #12] Loss: 1.0299 TOP1ACC: 64.4000% TOP5ACC: 9

Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a4e25b925f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a4e25b925f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/

[Test #106] Loss: 0.5628 TOP1ACC: 79.9300% TOP5ACC: 98.2300% Time: 6980.4388s


...

[Test #115] Loss: 0.5623 TOP1ACC: 80.3200% TOP5ACC: 98.1500% Time: 7577.0172s


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a4e25b925f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():Exception ignored in: 
<function _MultiProcessingDataLoaderIter.__del__ at 0x7a4e25b925f0>
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    assert self._parent_pid == os.getpid(), 'can only test a child process'
    AssertionError: self._shutdown_workers()can only test a child process

  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
Exception ignored in:     <function _M

[Test #116] Loss: 0.5518 TOP1ACC: 80.5800% TOP5ACC: 98.3300% Time: 7643.5228s


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a4e25b925f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    if w.is_alive():
  File "/usr/lib/python3.10/multiprocessing/process.py", line 160, in is_alive
    assert self._parent_pid == os.getpid(), 'can only test a child process'
AssertionError: can only test a child process
Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x7a4e25b925f0>
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1478, in __del__
    self._shutdown_workers()
  File "/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py", line 1461, in _shutdown_workers
    Exception ignored in: if w.is_aliv

[Test #117] Loss: 0.5686 TOP1ACC: 79.8900% TOP5ACC: 98.1900% Time: 7710.6775s
